# Model Training with nnfabrik

In [2]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nnfabrik
from nnfabrik.builder import get_data, get_model, get_trainer
import cascade


import datajoint as dj
dj.config["database.host"] = '18.198.155.36'

dj.config["enable_python_native_blobs"] = True
dj.config['nnfabrik.schema_name'] = "nnfabrik_neural_prediction_challenge"

schema = dj.schema("nnfabrik_neural_prediction_challenge")

from nnfabrik.main import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Initial Addition of Tables for the Schema

In [10]:
Fabrikant().insert1(dict(fabrikant_name='kwilleke',
                         email="konstantin.willeke@gmail.com",
                         affiliation='sinzlab',
                         dj_username="kwilleke"))

In [11]:
Seed().insert([{'seed':1000}])
Seed()

seed Random seed that is passed to the model- and dataset-builder
1000


# Add Dataset

In [4]:
# the cascade mouse dataloader takes the files from the Preprocessed mouse table and stores them on the server under /data/mouse/toliaslab/static/. like the other datasets
filenames = ['static26645-2-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip', 'static26644-14-17-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip']

dataset_fn = 'cascade.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': False,
                 'include_eye_position': True,
                 'batch_size': 128,
                 'exclude': None,
                 'file_tree': True,
                 'scale': 0.5
                 }

dataloaders = get_data(dataset_fn, dataset_config)

/data/mouse/toliaslab/static/static26645-2-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6 exists already. Not unpacking /data/mouse/toliaslab/static/static26645-2-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip
/data/mouse/toliaslab/static/static26644-14-17-GrayImageNet-94c6ff995dac583098847cfecd43e7b6 exists already. Not unpacking /data/mouse/toliaslab/static/static26644-14-17-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip


In [5]:
Dataset().add_entry(dataset_fn, dataset_config, dataset_comment="Both Test Scans, 72x128px, w/o behavior")

{'dataset_fn': 'cascade.datasets.static_loaders',
 'dataset_hash': '9483f97db6cca8f6c9c8c917eae046eb'}

In [6]:
b = next(iter(dataloaders["train"]["26644-14-17"]))

In [7]:
b.images.shape

torch.Size([128, 1, 72, 128])

# Add Model

In [8]:
model_fn = 'cascade.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
               'stack': -1,
               'layers': 4,
               'input_kern': 9,
               'gamma_input': 6.3831,
               'gamma_readout': 0.0076,
               'hidden_dilation': 1,
               'hidden_kern': 7,
               'hidden_channels': 64,
               'depth_separable': True,
               'init_sigma': 0.1,
               'init_mu_range': 0.3,
               'gauss_type': 'full',
               'shifter': True,
               'shift_layers': 3,
               'depth_separable': True,
               }

model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=1,)

/notebooks/ml-utils/neuralpredictors/layers/readouts/base.py:62: UserWarning: Use of 'gamma_readout' is deprecated. Please consider using the readout's feature-regularization parameter instead
  warnings.warn(
/notebooks/ml-utils/neuralpredictors/layers/readouts/base.py:79: UserWarning: Readout is NOT initialized with mean activity but with 0!
  warnings.warn("Readout is NOT initialized with mean activity but with 0!")


In [9]:
Model().add_entry(model_fn, model_config, model_comment="color mei default model")

{'model_fn': 'cascade.models.stacked_core_full_gauss_readout',
 'model_hash': 'd2664a1ba94dcd1d911c1ac6907934ec'}

# Add Trainer

In [10]:
trainer_fn = "cascade.training.standard_trainer"

trainer_config = {'max_iter': 2,
                 'verbose': False,
                 'lr_decay_steps': 4,
                 'avg_loss': False,
                 'lr_init': 0.009,
                 }

trainer = get_trainer(trainer_fn=trainer_fn, 
                     trainer_config=trainer_config)

In [11]:
Trainer().add_entry(trainer_fn, trainer_config, trainer_comment="color mei default trainer")

{'trainer_fn': 'cascade.training.standard_trainer',
 'trainer_hash': '534a2b5c34a223df3479adadd9f326ac'}

In [12]:
from nnvision.tables.from_nnfabrik import TrainedModel

In [ ]:
TrainedModel().populate(display_progress=True)

  0%|          | 0/1 [00:00<?, ?it/s]

/data/mouse/toliaslab/static/static26645-2-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6 exists already. Not unpacking /data/mouse/toliaslab/static/static26645-2-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip
/data/mouse/toliaslab/static/static26644-14-17-GrayImageNet-94c6ff995dac583098847cfecd43e7b6 exists already. Not unpacking /data/mouse/toliaslab/static/static26644-14-17-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip


/notebooks/ml-utils/neuralpredictors/layers/readouts/base.py:62: UserWarning: Use of 'gamma_readout' is deprecated. Please consider using the readout's feature-regularization parameter instead
  warnings.warn(
/notebooks/ml-utils/neuralpredictors/layers/readouts/base.py:79: UserWarning: Readout is NOT initialized with mean activity but with 0!
  warnings.warn("Readout is NOT initialized with mean activity but with 0!")

Epoch 1: 100%|██████████| 72/72 [01:44<00:00,  1.45s/it]


[001|00/05] ---> 0.05924167484045029



Epoch 2:  43%|████▎     | 31/72 [00:35<00:47,  1.16s/it]

In [ ]:
dataloaders, model = TrainedModel().load_model()